# Objetivo del notebook
Se consideran los datos de 2020 a 2023, se cree que en el llenado del formularios, los alumnos ingresaron de forma erronea su CP, por lo que usando su dirección se busco la que posiblemente se su CP correcta, generando poligonos y tomando las cordenadas del cetro de cada CP. 

El reto afrontar ahora es contrastar las CP obtenidas con las CP dadas por los alumnos o aspirantes.

## Formas a proceder.
* Se verificara la relevancia que representan las CP faltantes junto con los CP obtenidos 

In [1]:
import pandas as pd
import pickle


In [2]:
DATA_DIR = "../DataSets_Iniciales/coordenadas_obtenidas_google_maps/"

In [3]:

datasets = {
    "df_coordenadas_sustentantes_2020": pd.read_csv(f"{DATA_DIR}coordenadas_sustentantes_2020.csv"),
    "df_coordenadas_sustentantes_2021": pd.read_csv(f"{DATA_DIR}coordenadas_sustentantes_2021.csv"),
    "df_coordenadas_sustentantes_2022": pd.read_csv(f"{DATA_DIR}coordenadas_sustentantes_2022.csv"),
    "df_coordenadas_sustentantes_2023": pd.read_csv(f"{DATA_DIR}coordenadas_sustentantes_2023.csv")
}


In [4]:
#Saber cuantos valores distintos hay SUS_ENT
for key, df in datasets.items():
    print(f"Cantidad de valores distintos en {key}: {df.NOM_ENT.nunique()}")
# Guardar la cantidad de valores distintos en una lista


Cantidad de valores distintos en df_coordenadas_sustentantes_2020: 2
Cantidad de valores distintos en df_coordenadas_sustentantes_2021: 2
Cantidad de valores distintos en df_coordenadas_sustentantes_2022: 1
Cantidad de valores distintos en df_coordenadas_sustentantes_2023: 2


In [5]:
tipo_de_dato = datasets["df_coordenadas_sustentantes_2020"].postal_code_api.dtype
print(f"Tipo de dato de postal_code_api: {tipo_de_dato}")
tipo_de_dato_sus_cp = datasets["df_coordenadas_sustentantes_2020"].SUS_CP.dtype
print(f"Tipo de dato de SUS_CP: {tipo_de_dato_sus_cp}")

Tipo de dato de postal_code_api: float64
Tipo de dato de SUS_CP: int64


In [6]:
# Analizar valores nulos en postal_code_api para cada dataset
for name, df in datasets.items():
    # Total de filas
    total_rows = len(df)
    
    # Contar valores nulos en la columna postal_code_api
    null_count = df["postal_code_api"].isna().sum()
    
    # Calcular el porcentaje
    null_percentage = (null_count / total_rows) * 100
    
    print(f"{name}:")
    print(f"  Total de registros: {total_rows}")
    print(f"  Valores vacíos en postal_code_api: {null_count}")
    print(f"  Porcentaje de valores vacíos: {null_percentage:.2f}%")
    print(f"  Porcentaje de valores no vacíos: {100 - null_percentage:.2f}%")
    print()

df_coordenadas_sustentantes_2020:
  Total de registros: 25709
  Valores vacíos en postal_code_api: 4832
  Porcentaje de valores vacíos: 18.79%
  Porcentaje de valores no vacíos: 81.21%

df_coordenadas_sustentantes_2021:
  Total de registros: 24963
  Valores vacíos en postal_code_api: 4664
  Porcentaje de valores vacíos: 18.68%
  Porcentaje de valores no vacíos: 81.32%

df_coordenadas_sustentantes_2022:
  Total de registros: 13796
  Valores vacíos en postal_code_api: 2791
  Porcentaje de valores vacíos: 20.23%
  Porcentaje de valores no vacíos: 79.77%

df_coordenadas_sustentantes_2023:
  Total de registros: 18693
  Valores vacíos en postal_code_api: 2876
  Porcentaje de valores vacíos: 15.39%
  Porcentaje de valores no vacíos: 84.61%



In [7]:
#Crear un nuevo diccionario con DataFrames en donde solo esten los datos nulos de postal_code_api
datasets_nulos = {
    name: df[df["postal_code_api"].isna()] for name, df in datasets.items()
}

In [8]:
for name, df in datasets_nulos.items():
    total_rows = len(df)
    print(f"{name}:")
    print(f"  Total de registros con postal_code_api nulo: {total_rows}")

df_coordenadas_sustentantes_2020:
  Total de registros con postal_code_api nulo: 4832
df_coordenadas_sustentantes_2021:
  Total de registros con postal_code_api nulo: 4664
df_coordenadas_sustentantes_2022:
  Total de registros con postal_code_api nulo: 2791
df_coordenadas_sustentantes_2023:
  Total de registros con postal_code_api nulo: 2876


In [9]:
# Crear un nuevo diccionario con DataFrames sin valores faltantes en postal_code_api
datasets_clean = {
    name: df.dropna(subset=["postal_code_api"])
    for name, df in datasets.items()
}

In [10]:
for name, df in datasets_clean.items():
  df["postal_code_api"] = df["postal_code_api"].astype(int)

/tmp/ipykernel_136753/3430428774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["postal_code_api"] = df["postal_code_api"].astype(int)
/tmp/ipykernel_136753/3430428774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["postal_code_api"] = df["postal_code_api"].astype(int)
/tmp/ipykernel_136753/3430428774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [11]:

# Mostrar información sobre el proceso de limpieza
for name in datasets:
    original_rows = len(datasets[name])
    clean_rows = len(datasets_clean[name])
    removed_rows = original_rows - clean_rows
    
    print(f"{name}:")
    print(f"  Filas originales: {original_rows}")
    print(f"  Filas eliminadas: {removed_rows}")
    print(f"  Filas restantes: {clean_rows}")
    print(f"  Porcentaje de datos conservados: {(clean_rows/original_rows)*100:.2f}%")
    print()

df_coordenadas_sustentantes_2020:
  Filas originales: 25709
  Filas eliminadas: 4832
  Filas restantes: 20877
  Porcentaje de datos conservados: 81.21%

df_coordenadas_sustentantes_2021:
  Filas originales: 24963
  Filas eliminadas: 4664
  Filas restantes: 20299
  Porcentaje de datos conservados: 81.32%

df_coordenadas_sustentantes_2022:
  Filas originales: 13796
  Filas eliminadas: 2791
  Filas restantes: 11005
  Porcentaje de datos conservados: 79.77%

df_coordenadas_sustentantes_2023:
  Filas originales: 18693
  Filas eliminadas: 2876
  Filas restantes: 15817
  Porcentaje de datos conservados: 84.61%



In [12]:
tipo_de_dato = datasets_clean["df_coordenadas_sustentantes_2020"].postal_code_api.dtype
print(f"Tipo de dato de postal_code_api: {tipo_de_dato}")
tipo_de_dato_sus_cp = datasets_clean["df_coordenadas_sustentantes_2020"].SUS_CP.dtype
print(f"Tipo de dato de SUS_CP: {tipo_de_dato_sus_cp}")


Tipo de dato de postal_code_api: int64
Tipo de dato de SUS_CP: int64


In [13]:
for name, df in datasets_clean.items():
    
    # Total de registros
    total = len(df)
    
    # Contar coincidencias y diferencias
    coincidencias = (df["SUS_CP"] == df["postal_code_api"]).sum()
    diferencias = total - coincidencias
    
    # Calcular porcentajes
    porc_coincidencias = (coincidencias / total) * 100
    porc_diferencias = (diferencias / total) * 100
    
    print(f"{name}:")
    print(f"  Total de registros: {total}")
    print(f"  Códigos postales coincidentes: {coincidencias} ({porc_coincidencias:.2f}%)")
    print(f"  Códigos postales diferentes: {diferencias} ({porc_diferencias:.2f}%)")
    print()

df_coordenadas_sustentantes_2020:
  Total de registros: 20877
  Códigos postales coincidentes: 16347 (78.30%)
  Códigos postales diferentes: 4530 (21.70%)

df_coordenadas_sustentantes_2021:
  Total de registros: 20299
  Códigos postales coincidentes: 16164 (79.63%)
  Códigos postales diferentes: 4135 (20.37%)

df_coordenadas_sustentantes_2022:
  Total de registros: 11005
  Códigos postales coincidentes: 8509 (77.32%)
  Códigos postales diferentes: 2496 (22.68%)

df_coordenadas_sustentantes_2023:
  Total de registros: 15817
  Códigos postales coincidentes: 12351 (78.09%)
  Códigos postales diferentes: 3466 (21.91%)



In [14]:
datasets_iguales = {
    name: df[df["postal_code_api"] == df["SUS_CP"]] for name, df in datasets_clean.items()
}

In [15]:
datasets_diferentes = {
    name: df[df["postal_code_api"] != df["SUS_CP"]] for name, df in datasets_clean.items()
}

In [16]:
DATA_DIR_CLEAN = "../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas/"

In [17]:
# Guardar los DataFrames con valores nulos en un archivo CSV
for name, df in datasets_nulos.items():
    output_file = f"{DATA_DIR_CLEAN}/nulos/{name}_nulos.csv"
    df.to_csv(output_file, index=False)
    print(f"Datos nulos guardados en: {output_file}")

Datos nulos guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//nulos/df_coordenadas_sustentantes_2020_nulos.csv
Datos nulos guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//nulos/df_coordenadas_sustentantes_2021_nulos.csv
Datos nulos guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//nulos/df_coordenadas_sustentantes_2022_nulos.csv
Datos nulos guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//nulos/df_coordenadas_sustentantes_2023_nulos.csv


In [18]:
# Guardar los DataFrames con valores diferentes en un archivo CSV
for name, df in datasets_diferentes.items():
    output_file = f"{DATA_DIR_CLEAN}/diferentes/{name}_diferentes.csv"
    df.to_csv(output_file, index=False)
    print(f"Datos diferentes guardados en: {output_file}")

Datos diferentes guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//diferentes/df_coordenadas_sustentantes_2020_diferentes.csv
Datos diferentes guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//diferentes/df_coordenadas_sustentantes_2021_diferentes.csv
Datos diferentes guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//diferentes/df_coordenadas_sustentantes_2022_diferentes.csv
Datos diferentes guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//diferentes/df_coordenadas_sustentantes_2023_diferentes.csv


In [19]:
# Guardar los DataFrames con valores iguales en un archivo CSV
for name, df in datasets_iguales.items():
    output_file = f"{DATA_DIR_CLEAN}/iguales/{name}_iguales.csv"
    df.to_csv(output_file, index=False)
    print(f"Datos iguales guardados en: {output_file}")

Datos iguales guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//iguales/df_coordenadas_sustentantes_2020_iguales.csv
Datos iguales guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//iguales/df_coordenadas_sustentantes_2021_iguales.csv
Datos iguales guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//iguales/df_coordenadas_sustentantes_2022_iguales.csv
Datos iguales guardados en: ../DataSets_Iniciales/coordenadas_obtenidas_google_maps/Coordernadas_obtenidas_clasificadas//iguales/df_coordenadas_sustentantes_2023_iguales.csv
